# Analyse de Clustering avec GMM sur OpenFoodFacts
### Table des matières
1. [Chargement et préparation des données](#1),
2. [Prétraitement et nettoyage](#2),
3. [Sélection des features pour le clustering](#3),
4. [Préparation pour GMM](#4),
5. [Modélisation GMM](#5),
6. [Analyse des clusters](#6),
7. [Visualisation des résultats](#7)

## Chargement et préparation des données


In [1]:
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML

from scripts.features.processing import (
    analyze_correlations,
    process_numeric_columns,
    process_categorical_columns,
    detect_ordinal_nature,
    optimize_categorical_memory,
    handle_rare_categories,
    analyze_category_distributions,
    memory_usage_report,
    enhance_ordinal_detection,
    infer_ordinal_order
)
from scripts.features.selection import (
    analyze_and_select_features,
    analyze_feature_importance,
    select_relevant_features
)
from scripts.data.analysis import analyze_data_quality

# Chargement des données avec limitation à 300000 lignes
from scripts.data_handling.loader import load_data
df = pd.read_csv('../data/en.openfoodfacts.org.products.csv', 
                 sep='\t',
                 encoding='utf-8',
                 low_memory=False,
                 on_bad_lines='skip',
                 nrows=300000)

In [2]:
# Paramètres de prétraitement
MAX_CATEGORIES = 30  # Nombre maximum de catégories pour les variables catégorielles
MIN_UNIQUE_RATIO = 0.01  # Ratio minimum de valeurs uniques pour les variables numériques
MISSING_THRESHOLD = 0.5  # Seuil maximum de valeurs manquantes acceptables

# Analyse initiale de la qualité des données
print("1. Analyse de la qualité des données initiales :")
quality_report = analyze_data_quality(df)
print(f"\nNombre de lignes dupliquées : {quality_report['duplicates']}")
print("\nDistribution des valeurs manquantes :")
display(quality_report['missing_values'])

# Prétraitement et sélection automatique des features
print("\n2. Détection et prétraitement automatique des variables :")
df_processed, feature_info = analyze_and_select_features(
    df,
    max_categories=MAX_CATEGORIES,
    min_unique_ratio=MIN_UNIQUE_RATIO,
    missing_threshold=MISSING_THRESHOLD
)

# Affichage des résultats du prétraitement
print("\n3. Résumé des variables détectées :")
print(f"Variables numériques : {len(feature_info['feature_types']['numeric'])}")
print(f"Variables ordinales : {len(feature_info['feature_types']['ordinal'])}")
print(f"Variables nominales : {len(feature_info['feature_types']['nominal'])}")

# Affichage des optimisations de types de données
if feature_info['downcasted_columns']:
    print("\n4. Optimisations des types de données :")
    for col, old_type, new_type in feature_info['downcasted_columns']:
        print(f"- {col}: {old_type} → {new_type}")

# Affichage des colonnes filtrées
if feature_info['dropped_columns']:
    print("\n5. Colonnes filtrées :")
    for col, reason in feature_info['dropped_columns']:
        print(f"- {col} (raison: {reason})")

# Analyse des corrélations entre variables numériques
print("\n6. Analyse des corrélations entre variables numériques :")
correlations = analyze_correlations(df_processed, threshold=0.7, plot=True)
if correlations['strong_correlations']:
    print("\nPaires de variables fortement corrélées :")
    for corr in correlations['strong_correlations']:
        print(f"- {corr['var1']} - {corr['var2']}: {corr['correlation']:.2f}")

# Sauvegarde du DataFrame prétraité
df = df_processed.copy()
print(f"\nDimensions finales du DataFrame : {df.shape}")

# Nouvelles analyses avec les fonctionnalités ajoutées
print("\n7. Analyse approfondie des variables catégorielles :")

# Optimisation de la mémoire
print("\nOptimisation de la mémoire pour les variables catégorielles :")
df_optimized, memory_info = optimize_categorical_memory(df)
for col, savings in memory_info['memory_savings'].items():
    print(f"- {col}: {savings['savings_percent']:.2f}% d'économie de mémoire")

# Analyse des distributions des catégories
print("\nAnalyse des distributions des catégories :")
category_distributions = analyze_category_distributions(df)
for col, dist_info in category_distributions.items():
    print(f"\nColonne {col}:")
    print(f"- Nombre de catégories: {dist_info['n_categories']}")
    print(f"- Ratio de déséquilibre: {dist_info['imbalance_ratio']:.2f}")
    print(f"- Entropie: {dist_info['entropy']:.2f}")
    print("- Top 5 catégories les plus fréquentes:")
    for cat, count in dist_info['most_common'].items():
        print(f"  * {cat}: {count}")

# Gestion des catégories rares
print("\nGestion des catégories rares (seuil de 1%) :")
df_processed_rare, rare_info = handle_rare_categories(df, threshold=0.01)
for col, info in rare_info['modified_columns'].items():
    print(f"\nColonne {col}:")
    print(f"- Catégories initiales: {info['initial_categories']}")
    print(f"- Catégories finales: {info['final_categories']}")
    if col in rare_info['grouped_categories']:
        print(f"- Nombre de catégories regroupées: {rare_info['grouped_categories'][col]['n_grouped']}")

# Rapport d'utilisation de la mémoire
print("\n8. Rapport d'utilisation de la mémoire :")
memory_report = memory_usage_report(df)
print(f"Mémoire totale: {memory_report['total_memory_mb']:.2f} MB")
print("\nSuggestions d'optimisation:")
for suggestion in memory_report['optimization_suggestions']:
    print(f"- {suggestion}")

# Mise à jour du DataFrame avec les optimisations
df = df_optimized.copy()
print(f"\nDimensions finales après optimisation : {df.shape}")

1. Analyse de la qualité des données initiales :
⚠️ 196 colonnes sur 206 ont des valeurs manquantes.
allergens_en               100.000000
additives                  100.000000
cities                     100.000000
unsaturated-fat_100g       100.000000
casein_100g                100.000000
                              ...    
cholesterol_100g            58.607667
nutriscore_score            53.245667
nutrition-score-fr_100g     53.245667
nutrient_levels_tags        50.911333
nova_group                  50.390000
Length: 157, dtype: float64

Nombre de lignes dupliquées : 0

Distribution des valeurs manquantes :


allergens_en             100.000000
additives                100.000000
cities                   100.000000
unsaturated-fat_100g     100.000000
casein_100g              100.000000
                            ...    
pnns_groups_1              0.316667
pnns_groups_2              0.316667
completeness               0.004333
last_updated_t             0.000333
last_updated_datetime      0.000333
Length: 196, dtype: float64


2. Détection et prétraitement automatique des variables :
Colonnes numériques : 16
Colonnes ordinales : 1
Colonnes nominales : 1
Colonnes supprimées : 188
Colonnes optimisées : 2

3. Résumé des variables détectées :
Variables numériques : 16
Variables ordinales : 1
Variables nominales : 1

4. Optimisations des types de données :
- created_t: int64 → int32
- last_modified_t: int64 → int32

5. Colonnes filtrées :
- url (raison: too_many_categories)
- creator (raison: too_many_categories)
- created_datetime (raison: too_many_categories)
- last_modified_datetime (raison: too_many_categories)
- last_modified_by (raison: too_many_categories)
- last_updated_datetime (raison: too_many_categories)
- product_name (raison: too_many_categories)
- abbreviated_product_name (raison: too_many_missing)
- generic_name (raison: too_many_missing)
- quantity (raison: too_many_missing)
- packaging (raison: too_many_missing)
- packaging_tags (raison: too_many_missing)
- packaging_en (raison: too_many_missin